# Exploración de Datos Curados F1

Este notebook explora los datos curados del proyecto F1 Pitstop, genera insights sobre el rendimiento de pilotos y crea visualizaciones adicionales.

## 1. Cargar Datos Curados

Cargamos los datos desde el directorio `curated` utilizando pandas para leer archivos Parquet.

In [ ]:
from pathlib import Path

import pandas as pd
import plotly.express as px

In [ ]:
# Cargar datos de Bahrain Practice 1

# Ruta al directorio de datos
data_dir = Path("curated/track=Bahrain/session=Practice 1")

# Lista para almacenar DataFrames
dfs = []

# Iterar sobre las carpetas de pilotos
for driver_dir in data_dir.iterdir():
    if driver_dir.is_dir():
        parquet_file = driver_dir / "laps.parquet"
        if parquet_file.exists():
            df = pd.read_parquet(parquet_file)
            dfs.append(df)

# Concatenar todos los DataFrames
df_bahrain_p1: pd.DataFrame = pd.concat(dfs, ignore_index=True)

# Crear columna de nombre completo del piloto
df_bahrain_p1['driver_full_name'] = df_bahrain_p1['driverFirstName'] + ' ' + df_bahrain_p1['driverLastName']

## 2. Explorar Estructura de los Datos

Examinamos las columnas, tipos de datos y estadísticas básicas de los datos curados.

In [ ]:
# Explorar estructura de los datos
print("Columnas principales:")
main_cols = ['timestamp', 'trackName', 'sessionType', 'driverNumber', 'driverFirstName', 'driverLastName',
             'currentLap', 'lap_time_s', 'position', 'compound', 'tire_age', 'fuel', 'pace_index']
for col in main_cols:
    if col in df_bahrain_p1.columns:  # noqa: F821
        print(f"- {col}: {df_bahrain_p1[col].dtype}")  # noqa: F821

print("\nEstadísticas descriptivas:")
print(df_bahrain_p1[['lap_time_s', 'fuel', 'tire_age', 'pace_index']].describe())  # noqa: F821

print("\nDistribución de compuestos de neumáticos:")
print(df_bahrain_p1['compound'].value_counts())  # noqa: F821

print("\nRango de datos temporales:")
print(f"Inicio: {df_bahrain_p1['timestamp'].min()}")  # noqa: F821
print(f"Fin: {df_bahrain_p1['timestamp'].max()}")  # noqa: F821

Columnas principales:
- timestamp: datetime64[ns]
- trackName: object
- sessionType: object
- driverNumber: int64
- driverFirstName: object
- driverLastName: object
- currentLap: int64
- lap_time_s: float64
- position: int64
- compound: object
- tire_age: int64
- fuel: float64
- pace_index: float64

Estadísticas descriptivas:
       lap_time_s       fuel   tire_age  pace_index
count   56.000000  58.000000  58.000000   56.000000
mean   160.197679  15.901360  14.068966   22.035444
std    204.330492   7.001144   8.559107   28.105982
min      7.270000   2.416172   0.000000    1.000000
25%     96.896500   9.819930   7.000000   13.328267
50%     97.307000  17.079040  14.000000   13.384732
75%     98.008750  22.096010  21.000000   13.481259
max    929.101000  26.270330  30.000000  127.799312

Distribución de compuestos de neumáticos:
compound
Hard    58
Name: count, dtype: int64

Rango de datos temporales:
Inicio: 2025-08-24 01:16:57.683000
Fin: 2025-08-24 02:36:01.757000


## 3. Analizar Rendimiento de Pilotos

Calculamos métricas de rendimiento como tiempos de vuelta, posiciones y estadísticas agregadas por piloto.

In [ ]:
# Análisis de rendimiento por piloto
performance_stats = df_bahrain_p1.groupby('driver_full_name').agg({  # noqa: F821
    'lap_time_s': ['mean', 'std', 'min', 'max', 'count'],
    'position': ['mean', 'min', 'max'],
    'fuel': ['min', 'max'],
    'pace_index': ['mean', 'std']
}).round(3)

print("Estadísticas de rendimiento por piloto:")
performance_stats.columns = ['_'.join(col).strip() for col in performance_stats.columns.values]
performance_stats = performance_stats.reset_index()
print(performance_stats)

# Mejor vuelta por piloto
best_laps = df_bahrain_p1.loc[df_bahrain_p1.groupby('driver_full_name')['lap_time_s'].idxmin()]  # noqa: F821
print("\nMejores vueltas por piloto:")
print(best_laps[['driver_full_name', 'currentLap', 'lap_time_s', 'compound', 'tire_age', 'fuel']])

# Consistencia (coeficiente de variación de tiempos de vuelta)
consistency = df_bahrain_p1.groupby('driver_full_name')['lap_time_s'].std() / df_bahrain_p1.groupby('driver_full_name')['lap_time_s'].mean()  # noqa: F821
print("\nConsistencia (coeficiente de variación - menor es mejor):")
print(consistency.sort_values())

Estadísticas de rendimiento por piloto:
  driver_full_name  lap_time_s_mean  lap_time_s_std  lap_time_s_min  \
0  Fernando Alonso          158.136         187.817          61.203   
1     Lance Stroll          162.577         225.659           7.270   

   lap_time_s_max  lap_time_s_count  position_mean  position_min  \
0         928.267                30          9.000             2   
1         929.101                26         12.778             3   

   position_max  fuel_min  fuel_max  pace_index_mean  pace_index_std  
0            20     4.350    26.192           21.752          25.835  
1            20     2.416    26.270           22.363          31.040  

Mejores vueltas por piloto:
   driver_full_name  currentLap  lap_time_s compound  tire_age       fuel
30  Fernando Alonso          31      61.203     Hard        30  18.776150
57     Lance Stroll          27       7.270     Hard        26   2.416172

Consistencia (coeficiente de variación - menor es mejor):
driver_full_name
F

## 4. Generar Insights sobre Rendimiento

Aplicamos análisis estadísticos para identificar patrones y insights en el rendimiento de pilotos.

In [ ]:
# Insights sobre rendimiento

# 1. Análisis de degradación de neumáticos
tire_degradation = df_bahrain_p1.groupby(['driver_full_name', 'tire_age'])['lap_time_s'].mean().reset_index()  # noqa: F821

print("Insight 1: Degradación de neumáticos por piloto")
for driver in df_bahrain_p1['driver_full_name'].unique():  # type: ignore  # noqa: F821
    driver_data = tire_degradation[tire_degradation['driver_full_name'] == driver]
    if len(driver_data) > 1:
        initial_time = driver_data['lap_time_s'].iloc[0]  # type: ignore  # noqa: F821
        final_time = driver_data['lap_time_s'].iloc[-1]
        degradation = final_time - initial_time
        print(f"{driver}: Degradación total = {degradation:.3f}s ({'mejora' if degradation < 0 else 'empeoramiento'})")

# 2. Correlación entre combustible y rendimiento
fuel_performance_corr = df_bahrain_p1.groupby('driver_full_name').apply(  # noqa: F821
    lambda x: x['fuel'].corr(x['lap_time_s'])
).round(3)

print("\nInsight 2: Correlación entre nivel de combustible y tiempo de vuelta")
print("(Valores negativos indican que menos combustible = mejor tiempo)")
print(fuel_performance_corr)

# 3. Análisis de outliers en rendimiento
q1 = df_bahrain_p1['lap_time_s'].quantile(0.25)  # noqa: F821
q3 = df_bahrain_p1['lap_time_s'].quantile(0.75)  # noqa: F821
iqr = q3 - q1
outlier_threshold = q3 + 1.5 * iqr

outliers = df_bahrain_p1[df_bahrain_p1['lap_time_s'] > outlier_threshold]  # noqa: F821
print(f"\nInsight 3: Vueltas atípicas (outliers) detectadas: {len(outliers)}")
if len(outliers) > 0:
    print("Pilotos con outliers:")
    print(outliers.groupby('driver_full_name').size())

# 4. Eficiencia energética
energy_efficiency = df_bahrain_p1.groupby('driver_full_name')[['energyHarvested', 'energySpent']].mean()  # noqa: F821
energy_efficiency['net_energy'] = energy_efficiency['energyHarvested'] - energy_efficiency['energySpent']

print("\nInsight 4: Eficiencia energética (energía neta por piloto)")
print(energy_efficiency['net_energy'].sort_values(ascending=False))

Insight 1: Degradación de neumáticos por piloto
Fernando Alonso: Degradación total = nans (empeoramiento)
Lance Stroll: Degradación total = nans (empeoramiento)

Insight 2: Correlación entre nivel de combustible y tiempo de vuelta
(Valores negativos indican que menos combustible = mejor tiempo)
driver_full_name
Fernando Alonso    0.024
Lance Stroll       0.116
dtype: float64

Insight 3: Vueltas atípicas (outliers) detectadas: 8
Pilotos con outliers:
driver_full_name
Fernando Alonso    4
Lance Stroll       4
dtype: int64

Insight 4: Eficiencia energética (energía neta por piloto)
driver_full_name
Fernando Alonso   -0.021156
Lance Stroll      -0.024020
Name: net_energy, dtype: float64


C:\Users\doski\AppData\Local\Temp\ipykernel_13784\3964935644.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fuel_performance_corr = df_bahrain_p1.groupby('driver_full_name').apply(


## 5. Crear Visualizaciones Adicionales

Generamos gráficos y visualizaciones usando Plotly para ilustrar los insights encontrados.

In [ ]:
# Visualización 1: Comparación de tiempos de vuelta por piloto
fig1 = px.line(df_bahrain_p1, x='currentLap', y='lap_time_s', color='driver_full_name',  # noqa: F821  # type: ignore[reportUndefinedVariable]
               title='Comparación de Tiempos de Vuelta - Practice 1 Bahrain',
               labels={'currentLap': 'Vuelta', 'lap_time_s': 'Tiempo de Vuelta (s)', 'driver_full_name': 'Piloto'})
fig1.update_layout(height=500)
fig1.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Visualización 2: Degradación de neumáticos
fig2 = px.line(tire_degradation, x='tire_age', y='lap_time_s', color='driver_full_name',  # noqa: F821  # type: ignore[reportUndefinedVariable]
               title='Degradación de Neumáticos por Piloto',
               labels={'tire_age': 'Edad del Neumático', 'lap_time_s': 'Tiempo Promedio de Vuelta (s)', 'driver_full_name': 'Piloto'})
fig2.update_layout(height=500)
fig2.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Visualización 3: Consumo de combustible vs tiempo de vuelta
fig3 = px.scatter(df_bahrain_p1, x='fuel', y='lap_time_s', color='driver_full_name',  # noqa: F821  # type: ignore[reportUndefinedVariable]
                  title='Relación entre Combustible y Tiempo de Vuelta',
                  labels={'fuel': 'Combustible Restante', 'lap_time_s': 'Tiempo de Vuelta (s)', 'driver_full_name': 'Piloto'},
                  trendline="ols")
fig3.update_layout(height=500)
fig3.show()

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
# Visualización 4: Distribución de tiempos de vuelta por piloto
fig4 = px.box(df_bahrain_p1, x='driver_full_name', y='lap_time_s',  # noqa: F821  # type: ignore[reportUndefinedVariable]
              title='Distribución de Tiempos de Vuelta por Piloto',
              labels={'driver_full_name': 'Piloto', 'lap_time_s': 'Tiempo de Vuelta (s)'})
fig4.update_layout(height=500)
fig4.show()

# Visualización 5: Posiciones a lo largo de la sesión
fig5 = px.line(df_bahrain_p1, x='currentLap', y='position', color='driver_full_name',  # noqa: F821  # type: ignore[reportUndefinedVariable]
               title='Evolución de Posiciones - Practice 1 Bahrain',
               labels={'currentLap': 'Vuelta', 'position': 'Posición', 'driver_full_name': 'Piloto'})
fig5.update_yaxes(autorange="reversed")  # Posición 1 arriba
fig5.update_layout(height=500)
fig5.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Resumen de Insights

Este notebook ha explorado los datos curados de F1 y generado varios insights importantes:

1. **Rendimiento General**: Comparación de tiempos de vuelta, posiciones y estadísticas por piloto.

2. **Degradación de Neumáticos**: Análisis de cómo los neumáticos se degradan con el uso y diferencias entre pilotos.

3. **Eficiencia de Combustible**: Relación entre el nivel de combustible restante y el rendimiento.

4. **Consistencia**: Medición de la variabilidad en los tiempos de vuelta.

5. **Eficiencia Energética**: Análisis del balance entre energía recuperada y gastada.

Las visualizaciones interactivas permiten explorar estos datos de manera detallada y comparar el rendimiento entre pilotos en diferentes aspectos de la carrera.